In [1]:
import os
import numpy as np
import pandas as pd
import json

# Response 2 and Demographics

In [2]:
raw_R2_and_D_datapath = "response_2_and_demographics_raw.csv"
processed_R2_and_D_datapath = "processed/response_2_and_demographics_processed.csv"

raw_columns = [ele.strip() for ele in pd.read_csv("headers/raw_header_phase_2_and_demo.csv").columns]

In [3]:
raw_R2_and_D_df = pd.read_csv(raw_R2_and_D_datapath, header=None)
raw_R2_and_D_df.columns = raw_columns
raw_R2_and_D_df.head(2)

userId  question_number                                      breed_key  \
0       2                0  dog_A : chihuahua,dog_B : miniature_pinscher,   
1       2                1  dog_A : chihuahua,dog_B : miniature_pinscher,   

  query_policy visualization_mode  \
0     policy_3     scatter_plot_1   
1     policy_3     scatter_plot_1   

                                          image_name   majority_label  \
0  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   
1  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   

   user_agreement  user_confidence  system_confidence     ...      \
0               5                5                  5     ...       
1               3                3                  3     ...       

   monitor_size  comments  userId.1  turkId           ip dataset         Vis  \
0             1       NaN         2     NaN  50.81.83.64   setB1  scatter_pl   
1             1       NaN         2     NaN  50.81.83.64   setB1  scatter_pl   

     policy       start         end  
0  policy_3  2018-03-22  2018-03-22  
1  policy_3  2018-03-22  2018-03-22  

[2 rows x 33 columns]

# 1. Normalize user response

In [4]:
raw_R2_and_D_df["user_response"] = raw_R2_and_D_df["user_response"] - 1

# 2. Remove Bad Users
User other script. Pull in valid user IDs. Create new DF from that list: valid_user_keys.csv

### Phase 2

#### Get Valid User IDs from phase 2

In [5]:
def get_breed_key(user_i):
    breed_key = list(user_i["breed_key"])[0]
    breed_key = { ele.split(" : ")[0] : ele.split(" : ")[1] for ele in breed_key.split(",") if ele != "" }
    return breed_key

In [6]:
def get_actual_labels(user_i, breed_key):
    image_names = [ele.split("/imgs/")[1] for ele in user_i["image_name"]]
    for img_i, img in enumerate(image_names):
        for key, val in breed_key.items():
            if key in img:
                image_names[img_i] = img.replace(key, val).split("/dogs/")[1].split("__")[0]
    return image_names

### Duplicate Users

In [7]:
all_user_ids = np.unique(raw_R2_and_D_df["userId"])
print("Num users: {}".format(len(all_user_ids)))

Num users: 637


In [8]:
datasets = ["setA1", "setA2", "setB1", "setB2", "setC1", "setC2"]
visualizations = ["none", "numeric", "bar_chart_1", "bar_chart_2", "scatter_plot_1", "scatter_plot_2", "tSNE"]

conditions_counter = {}
for policy_i in range(1, 6):
    conditions_counter["policy_{}".format(policy_i)] = {}
    for vis_i in visualizations:
        conditions_counter["policy_{}".format(policy_i)][vis_i] = {}
        for dataset_i in datasets:
            conditions_counter["policy_{}".format(policy_i)][vis_i][dataset_i] = 0
            

In [9]:
def update_condition_counter(user_i):
    policy = list(user_i["query_policy"])[0]
    vis = list(user_i["visualization_mode"])[0]
    dataset = list(user_i["dataset"])[0]
    conditions_counter[policy][vis][dataset] += 1

In [10]:
invalid_user_ids = []
missing_data = {}
actual_labels = []
user_groups = raw_R2_and_D_df.groupby("userId")
for i, user_group_i in enumerate(user_groups):
    invalid_user = False
    user_i = user_group_i[1]
    user_id = list(user_i.id)[0]
    
    # Duplicate users
    if len(user_i) != 25:
        invalid_user = True
        invalid_user_ids.append(user_id)
    
    breed_key = get_breed_key(user_i)
    user_actual_labels = get_actual_labels(user_i, breed_key)
    actual_labels += user_actual_labels
    
    # All Likert-type answers are the same
    user_self_conf = len(set(user_i["user_confidence"])) == 1
    user_agree = len(set(user_i["user_agreement"])) == 1
    user_system_conf = len(set(user_i["system_confidence"])) == 1
    
    # Filter on same answer for all likert questions
    if user_self_conf and user_agree and user_system_conf:
        invalid_user_ids.append(user_id)
        invalid_user = True
        
    if not invalid_user:
        update_condition_counter(user_i)
    

In [11]:
conditions_needed = []
for policy_i in range(1, 6):
    for vis_i in visualizations:
        for dataset_i in datasets:
            condition_count = conditions_counter["policy_{}".format(policy_i)][vis_i][dataset_i]
            if condition_count < 3:
                condition_needed = "policy_{}__{}__{}".format(policy_i, vis_i, dataset_i)
                conditions_needed.append(condition_needed)
num_conditions_needed = len(conditions_needed)
print("Num needed: {}".format(num_conditions_needed))

Num needed: 22


In [12]:
"""
"https://cu-visualab.org/oracle_query/S2/Policy1/Vis6/setC1"
"""
vis_lookup = {
    "none" : "Vis1",
    "numeric" : "Vis2",
    "bar_chart_1" : "Vis3",
    "bar_chart_2" : "Vis4",
    "scatter_plot_1" : "Vis5",
    "scatter_plot_2" : "Vis6",
    "tSNE" : "Vis7"
}

mech_turk_urls = []
for condition in conditions_needed:
    policy = condition.split("__")[0].replace("_", "").capitalize()
    vis = vis_lookup[condition.split("__")[1]]
    dataset = condition.split("__")[2]
    
    url = "https://cu-visualab.org/oracle_query/S2/{}/{}/{}".format(policy, vis, dataset)
    mech_turk_urls.append(url)

In [13]:
len(mech_turk_urls)

22

In [14]:
with open("mech_turk_urls.txt", "w") as outfile:
    for url_i, url in enumerate(mech_turk_urls):
        if url_i % 8 == 0:
            outfile.write("\n")
        outfile.write("{}\n".format(url))

## Get Missing Data

### Update raw_R2_and_D_df with actual labels

In [15]:
actual_labels_df = pd.DataFrame(actual_labels)
actual_labels_df.columns = ["actual_label"]

In [16]:
raw_R2_and_D_df = pd.concat([raw_R2_and_D_df, actual_labels_df], axis=1)
raw_R2_and_D_df.head(2)

userId  question_number                                      breed_key  \
0       2                0  dog_A : chihuahua,dog_B : miniature_pinscher,   
1       2                1  dog_A : chihuahua,dog_B : miniature_pinscher,   

  query_policy visualization_mode  \
0     policy_3     scatter_plot_1   
1     policy_3     scatter_plot_1   

                                          image_name   majority_label  \
0  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   
1  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   

   user_agreement  user_confidence  system_confidence      ...        \
0               5                5                  5      ...         
1               3                3                  3      ...         

   comments  userId.1  turkId           ip dataset         Vis    policy  \
0       NaN         2     NaN  50.81.83.64   setB1  scatter_pl  policy_3   
1       NaN         2     NaN  50.81.83.64   setB1  scatter_pl  policy_3   

        start         end   actual_label  
0  2018-03-22  2018-03-22   basset_hound  
1  2018-03-22  2018-03-22  saint_bernard  

[2 rows x 34 columns]

In [17]:
temp_valid_user_ids = list(set(all_user_ids) - set(invalid_user_ids))

### Phase 1

In [18]:
phase_1_path = "response_data_phase_1.csv"

In [19]:
phase_1_df = pd.read_csv(phase_1_path, header=None)
phase_1_df.columns = ["userId", "question_number", "breed_key", "query_policy", "visualization_mode",
            "image_name", "user_response", "majority_label", "response_time"]

In [20]:
phase_1_df_invalid_removed = phase_1_df[phase_1_df["userId"].isin(temp_valid_user_ids)]

In [21]:
phase_1_groups = phase_1_df_invalid_removed.groupby("userId")
for user_group_i in phase_1_groups:
    
    user_i = user_group_i[1]
    user_id = list(user_i.userId)[0]
    user_responses = user_i["user_response"]
    if len(np.unique(user_responses)) != 2:
        invalid_user_ids.append(user_id)

### Create Valid User Ids

In [22]:
valid_user_ids = list(set(all_user_ids) - set(invalid_user_ids))
print("Num valid users: {}".format(len(valid_user_ids)))

Num valid users: 603


### Save Valid User IDs

In [23]:
valid_user_ids_df = pd.DataFrame(valid_user_ids)
valid_user_ids_df.columns = ["userId"]
valid_user_ids_df.to_csv("valid_user_IDs.csv")

In [24]:
final_valid_users_df = raw_R2_and_D_df[raw_R2_and_D_df["userId"].isin(valid_user_ids)]
print("len(valid_user_ids) : {} -- len(raw_R2_and_D_df_cleaned) : {}".format(len(valid_user_ids), int(len(final_valid_users_df)/25.0)))

len(valid_user_ids) : 603 -- len(raw_R2_and_D_df_cleaned) : 603


In [25]:
final_valid_users_df.head(2)

userId  question_number                                      breed_key  \
0       2                0  dog_A : chihuahua,dog_B : miniature_pinscher,   
1       2                1  dog_A : chihuahua,dog_B : miniature_pinscher,   

  query_policy visualization_mode  \
0     policy_3     scatter_plot_1   
1     policy_3     scatter_plot_1   

                                          image_name   majority_label  \
0  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   
1  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   

   user_agreement  user_confidence  system_confidence      ...        \
0               5                5                  5      ...         
1               3                3                  3      ...         

   comments  userId.1  turkId           ip dataset         Vis    policy  \
0       NaN         2     NaN  50.81.83.64   setB1  scatter_pl  policy_3   
1       NaN         2     NaN  50.81.83.64   setB1  scatter_pl  policy_3   

        start         end   actual_label  
0  2018-03-22  2018-03-22   basset_hound  
1  2018-03-22  2018-03-22  saint_bernard  

[2 rows x 34 columns]

# 3. Get Raw Image Confidence

In [26]:
file_information_path = "condition_data/"
file_information_path

'condition_data/'

In [27]:
"""
Example: policy_1__bar_chart_1__setA1.json
"""

image_raw_confidences = []
for i, row_i in final_valid_users_df.iterrows():
    user_img = row_i["image_name"].split("/dogs/")[1].strip()
    user_policy = row_i["query_policy"]
    user_vis = row_i["visualization_mode"]
    user_dataset = row_i["dataset"]
    condition_reference = "condition_data/{}__{}__{}.json".format(user_policy, user_vis, user_dataset)

    with open(condition_reference) as infile:
        condition_info = json.load(infile)
    
    order = ["test", "training"]
    img_found = False
    for key_i in order:
        cond = condition_info[key_i]
        for condition_img in cond:
            img_path = condition_img["secure_path"].split("/dogs/")[1].strip()

            if img_path == user_img:
                img_found = True
                img_confidence = condition_img["img_confidence"]
                image_raw_confidences.append(img_confidence) 
                break
        
        if img_found:
            break

In [28]:
img_confidence_df = pd.DataFrame(image_raw_confidences)
img_confidence_df.columns = ["img_confidence"]

In [29]:
len(final_valid_users_df)

15075

In [30]:
len(img_confidence_df)

15075

In [31]:
final_df = pd.concat([final_valid_users_df, img_confidence_df], axis=1)

In [32]:
final_df.head(2)

userId  question_number                                      breed_key  \
0     2.0              0.0  dog_A : chihuahua,dog_B : miniature_pinscher,   
1     2.0              1.0  dog_A : chihuahua,dog_B : miniature_pinscher,   

  query_policy visualization_mode  \
0     policy_3     scatter_plot_1   
1     policy_3     scatter_plot_1   

                                          image_name   majority_label  \
0  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   
1  http:/cu-visualab.org/oracle_query/S2/imgs/set...  miniature_pinsc   

   user_agreement  user_confidence  system_confidence      ...        \
0             5.0              5.0                5.0      ...         
1             3.0              3.0                3.0      ...         

   userId.1  turkId           ip  dataset         Vis    policy       start  \
0       2.0     NaN  50.81.83.64    setB1  scatter_pl  policy_3  2018-03-22   
1       2.0     NaN  50.81.83.64    setB1  scatter_pl  policy_3  2018-03-22   

          end   actual_label img_confidence  
0  2018-03-22   basset_hound          0.875  
1  2018-03-22  saint_bernard          1.000  

[2 rows x 35 columns]

In [33]:
final_df.to_csv(processed_R2_and_D_datapath)

In [34]:
len(final_df) / 25.0

636.0

# Questionnaire and Demographics

In [35]:
raw_questionnaire_and_demographics_datapath = "questionnaire_and_demographics_raw.csv"
processed_questionnaire_and_demographics_datapath = "processed/questionnaire_and_demographics_processed.csv"

raw_questionnaire_and_demographics_columns = [ele.strip() for ele in pd.read_csv("headers/raw_header_questionnaire_and_demo.csv").columns]

In [36]:
raw_Q_and_D_df = pd.read_csv(raw_questionnaire_and_demographics_datapath, header=None)
raw_Q_and_D_df.columns = raw_questionnaire_and_demographics_columns
raw_Q_and_D_df.head(2)

userId  Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8  Q9     ...      monitor_size  \
0       2   4   3   4   4   3   2   3   3   5     ...                 1   
1       3   2   5   4   3   5   2   2   5   5     ...                13   

   comments  userID  turkID               ip  dataset         Vis    policy  \
0       NaN       2     NaN      50.81.83.64    setB1  scatter_pl  policy_3   
1       NaN       3     NaN  104.138.240.110    setC1        tSNE  policy_1   

        start         end  
0  2018-03-22  2018-03-22  
1  2018-03-22  2018-03-22  

[2 rows x 35 columns]

### Replace bad Vis column with good vis

In [76]:
user_vis = []
for i, row_i in raw_Q_and_D_df.iterrows():
    userId = row_i["userId"]
    user_i = raw_R2_and_D_df[raw_R2_and_D_df["userId"]==userId]
    try:
        vis = list(user_i["visualization_mode"])[0]
        user_vis.append(vis)
    except:
        continue

In [77]:
user_visualizations_df = pd.DataFrame(user_vis)
user_visualizations_df.columns = ["visualizations"]

In [78]:
raw_Q_and_D_df_test = pd.concat([raw_Q_and_D_df, user_visualizations_df], axis=1)

In [79]:
raw_Q_and_D_df_test

userId  Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8  Q9       ...        \
0         2   4   3   4   4   3   2   3   3   5       ...         
1         3   2   5   4   3   5   2   2   5   5       ...         
2         1   2   5   3   2   2   2   2   5   4       ...         
3         6   5   5   4   4   5   3   4   3   4       ...         
4         7   1   4   1   1   4   1   2   5   4       ...         
5         4   1   5   1   1   1   1   1   6   4       ...         
6         5   6   3   5   5   2   2   3   4   4       ...         
7         9   3   5   3   4   1   1   1   5   4       ...         
8        12   3   5   3   3   3   2   2   4   3       ...         
9         8   2   5   1   1   3   1   2   5   6       ...         
10       14   1   4   3   4   3   1   3   4   4       ...         
11       11   2   5   4   6   5   2   2   5   6       ...         
12       10   5   5   6   5   5   5   5   5   5       ...         
13       15   6   4   5   6   5   4   5   6   5       ...         
14       15   6   4   5   6   5   4   5   6   5       ...         
15       15   6   4   5   6   5   4   5   6   5       ...         
16       16   4   4   5   5   4   4   4   4   3       ...         
17       18   4   4   5   5   5   3   4   5   5       ...         
18       21   4   5   4   4   4   4   4   5   3       ...         
19       19   2   5   3   6   3   1   4   6   7       ...         
20       20   1   6   1   2   2   1   1   6   4       ...         
21       17   3   2   5   5   5   5   6   2   1       ...         
22       22   1   5   1   1   7   1   1   6   7       ...         
23       24   1   4   1   1   1   1   7   4   4       ...         
24       25   1   5   1   1   1   1   1   6   7       ...         
25       26   3   4   2   1   1   1   4   3   4       ...         
26       28   2   5   2   1   2   2   2   6   6       ...         
27       29   2   6   2   2   2   1   2   6   5       ...         
28       31   5   3   4   4   4   4   4   4   5       ...         
29       32   1   7   1   4   2   1   1   7   7       ...         
..      ...  ..  ..  ..  ..  ..  ..  ..  ..  ..       ...         
622    1015   2   5   3   3   2   2   2   6   4       ...         
623    1017   2   6   2   2   2   1   2   5   6       ...         
624    1021   3   6   2   2   3   2   1   6   6       ...         
625    1020   3   5   3   3   2   1   2   5   5       ...         
626    1019   5   6   4   4   5   4   5   4   5       ...         
627    1024   6   4   6   6   3   5   5   2   4       ...         
628    1028   2   5   3   3   2   2   2   6   6       ...         
629    1027   3   5   3   3   2   2   2   5   3       ...         
630    1030   1   7   1   1   1   1   1   7   5       ...         
631    1031   2   7   1   1   1   1   1   7   7       ...         
632    1034   6   2   7   7   5   7   6   2   2       ...         
633    1038   4   5   3   7   4   1   3   5   7       ...         
634    1039   3   3   5   5   4   4   3   5   2       ...         
635    1044   5   6   4   5   3   2   4   6   6       ...         
636    1045   4   5   2   2   4   4   3   4   1       ...         
637    1048   3   5   3   3   4   3   3   5   5       ...         
638    1047   3   3   3   5   4   2   3   5   3       ...         
639    1050   4   5   3   4   4   4   4   4   5       ...         
640    1049   4   5   4   5   6   3   4   5   6       ...         
641    1052   1   3   1   1   1   1   3   3   4       ...         
642    1052   1   3   1   1   1   1   3   3   4       ...         
643    1052   1   3   1   1   1   1   3   3   4       ...         
644    1052   1   3   1   1   1   1   3   3   4       ...         
645    1052   1   3   1   1   1   1   3   3   4       ...         
646    1052   1   3   1   1   1   1   3   3   4       ...         
647    1053   6   3   5   5   4   6   6   7   5       ...         
648    1054   2   6   2   3   3   2   2   5   5       ...         
649    1056   4   4   4   6   4   3   5   4   5     

In [80]:
raw_Q_and_D_df_test.to_csv(processed_questionnaire_and_demographics_datapath)